In [1]:
pip install bert-score


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
from pathlib import Path
from bert_score import BERTScorer

csv_path = Path("/home/vmadmin/intent/src/test/openai-gpt-5.csv")
raw_df = pd.read_csv(csv_path)

# Extract JSON-like tool arguments from the serialized response blob
def extract_arguments(payload):
    if not isinstance(payload, str):
        return None
    match = re.search(r"arguments='(.*?)',\s*call_id", payload, flags=re.DOTALL)
    if not match:
        return None
    return match.group(1).replace('""', '"')

intent_arguments_df = raw_df.assign(
    arguments=raw_df["intent_processing"].map(extract_arguments)
)[["intent", "arguments"]]

intent_arguments_df

/home/vmadmin/intent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,intent,arguments
0,Create a slice to support video journalists tr...,"{""body"":{""serviceTime"":null,""serviceArea"":null..."
1,Provision a slice for a university campus even...,"{""body"":{""serviceTime"":null,""serviceArea"":null..."
2,Set up a slice for a shopping mall to improve ...,"{""body"":{""serviceTime"":null,""serviceArea"":null..."
3,Deploy a slice for a media production crew usi...,"{""body"":{""serviceTime"":null,""serviceArea"":null..."
4,Establish a slice for commuters in a busy trai...,"{""body"":{""serviceTime"":null,""serviceArea"":null..."
...,...,...
95,Provision a slice for smart greenhouse climate...,"{""body"":{""serviceTime"":null,""serviceArea"":null..."
96,Establish a slice for public safety siren moni...,"{""body"":{""serviceTime"":null,""serviceArea"":null..."
97,Deploy a slice for high-resolution mobile mapp...,"{""body"":{""serviceTime"":null,""serviceArea"":null..."
98,Create a slice for autonomous ferry navigation...,"{""body"":{""serviceTime"":null,""serviceArea"":null..."


In [3]:
intent_arguments_df = intent_arguments_df.head(1)

for index, row in intent_arguments_df.iterrows():
    scorer = BERTScorer(
        model_type="microsoft/deberta-xlarge-mnli",
        lang="en",
        use_fast_tokenizer=True,
    )
    scorer._tokenizer.model_max_length = 512
    scorer._tokenizer.init_kwargs["model_max_length"] = 512  # keep future loads consistent

    P, R, F1 = scorer.score(row.intent, row.arguments)

    intent_arguments_df["precision"] = P.mean()
    intent_arguments_df["recall"] = R.mean()
    intent_arguments_df["F1 Score"] = F1.mean()

Loading weights: 100%|██████████| 772/772 [00:02<00:00, 359.65it/s, Materializing param=encoder.rel_embeddings.weight]                     
DebertaModel LOAD REPORT from: microsoft/deberta-xlarge-mnli
Key                 | Status     |  | 
--------------------+------------+--+-
classifier.bias     | UNEXPECTED |  | 
pooler.dense.bias   | UNEXPECTED |  | 
classifier.weight   | UNEXPECTED |  | 
pooler.dense.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


AttributeError: DebertaTokenizer has no attribute build_inputs_with_special_tokens